In [3]:
import os
import json
import warnings
import random

import dgl
import torch
import torch_geometric as tg

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import dgl.nn.pytorch as dglnn

from graphlime import GraphLIME
from mumin import MuminDataset

import torchmetrics as tm

from tqdm.notebook import tqdm
from collections import defaultdict

from torch_geometric.data import Data as tgData

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

from mumin_explainable.architectures.graphs import GAT
from mumin_explainable.processor.tweetnormalizer import normalizeTweet

from dotenv import load_dotenv

import mlflow


%matplotlib inline

warnings.filterwarnings('ignore')
_= torch.manual_seed(42)

# Setup mumin graph

In [4]:
size= 'small'

dataset_mumin = MuminDataset(
    twitter_bearer_token=os.getenv('TWITTER_BEARER_TOKEN'),
    size=size,
    dataset_path=f'./data/datasets/mumin-{size}.zip'
)
dataset_mumin.compile()
mumin_graph = dataset_mumin.to_dgl()
mumin_graph

2023-06-30 16:36:07,335 [INFO] Loading dataset


2023-06-30 16:36:29,579 [INFO] Outputting to DGL


Graph(num_nodes={'article': 1446, 'claim': 2083, 'hashtag': 27802, 'image': 1015, 'reply': 177816, 'tweet': 4061, 'user': 152038},
      num_edges={('article', 'has_article_inv', 'tweet'): 1890, ('claim', 'discusses_inv', 'tweet'): 4749, ('hashtag', 'has_hashtag_inv', 'tweet'): 2284, ('hashtag', 'has_hashtag_inv', 'user'): 49626, ('image', 'has_image_inv', 'tweet'): 1019, ('reply', 'posted_inv', 'user'): 177816, ('reply', 'quote_of', 'tweet'): 88495, ('reply', 'reply_to', 'tweet'): 78576, ('tweet', 'discusses', 'claim'): 4749, ('tweet', 'has_article', 'article'): 1890, ('tweet', 'has_hashtag', 'hashtag'): 2284, ('tweet', 'has_image', 'image'): 1019, ('tweet', 'mentions', 'user'): 1112, ('tweet', 'posted_inv', 'user'): 4061, ('tweet', 'quote_of_inv', 'reply'): 88495, ('tweet', 'reply_to_inv', 'reply'): 78576, ('tweet', 'retweeted_inv', 'user'): 12800, ('user', 'follows', 'user'): 17974, ('user', 'follows_inv', 'user'): 17974, ('user', 'has_hashtag', 'hashtag'): 49626, ('user', 'mentions

In [20]:
dataset_mumin.nodes['reply'].dropna()

,tweet_id,text,created_at,lang,source,num_retweets,num_replies,num_quote_tweets,text_emb,lang_emb
0,1238947475471454220,Antes de llegar a los pulmones dura 4 días en ...,2020-03-14 21:57:51,es,Twitter for Android,8,3,0,"[-0.0467078, 0.25795, 0.119816095, 0.4975067, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,1239109625200377858,@rtejedaguerrero https://t.co/hlZJUlZTp6,2020-03-15 08:42:10,und,Twitter for Android,1,0,0,"[-0.053789407, 0.24964282, 0.13200346, 0.50709...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1295197073034412033,N tem um minuto de paz https://t.co/2RNoCRV134,2020-08-17 03:13:40,pt,Twitter for Android,0,0,0,"[-0.04644562, 0.2476034, 0.12645161, 0.4954996...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1295196978868092928,cpa que já ta bom em 2020 https://t.co/2aJ68Mme1v,2020-08-17 03:13:18,pt,Twitter Web App,1,0,0,"[-0.057990123, 0.24166676, 0.12565383, 0.48768...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1295196650479259648,2020 tem q acabar logo https://t.co/IwdYPgjVqu,2020-08-17 03:11:59,pt,Twitter for iPhone,0,0,0,"[-0.056101725, 0.23295374, 0.1282421, 0.492408...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...
177811,1367507981034741770,"@ArtsenV Goed gelachen met ""annafylactische sh...",2021-03-04 16:11:24,nl,Twitter Web App,0,0,0,"[-0.052896257, 0.24198653, 0.114569515, 0.4974...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
177812,1367430006151012354,@ArtsenV Dat het fakenieuws wisten jullie nu a...,2021-03-04 11:01:33,nl,Twitter for iPhone,0,0,0,"[-0.04555127, 0.24387239, 0.110640965, 0.48637...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
177813,1367429518668009474,@kwoensj @ArtsenV Gans is beter.,2021-03-04 10:59:37,en,Twitter for iPhone,0,0,0,"[-0.04102891, 0.23638375, 0.11785056, 0.505451...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
177814,1367406762790424589,@ArtsenV Achterlijke gekken zijn jullie!,2021-03-04 09:29:12,nl,Twitter for Android,0,0,0,"[-0.03991937, 0.2260326, 0.10493929, 0.515906,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
plt.figure(figsize=(10, 7))
metagraph = mumin_graph.metagraph()
nx.draw_networkx(metagraph, 
                 pos=nx.shell_layout(metagraph), 
                 node_color='white', 
                 node_size=3000,
                 arrows=False)

In [ ]:
user_df = dataset_mumin.nodes['user']
tweet_df = dataset_mumin.nodes['tweet']

user_posted_tweet_df = dataset_mumin.rels[('user', 'posted', 'tweet')]

# user_df.dropna(inplace=True)
# claim_df.dropna(inplace=True)
# tweet_df.dropna(inplace=True)

# Setup subgraph

In [ ]:
user_posted_tweet_subgraph = dgl.edge_type_subgraph(mumin_graph, etypes=[('user', 'posted', 'tweet')])

## Filter language

In [ ]:
LANG = 'multilingual'
tweet_ds_copy = dataset_mumin.nodes['tweet'].dropna()
lang_tweets = (tweet_ds_copy['lang'] == LANG).to_list()
del(tweet_ds_copy)

In [ ]:
if LANG == 'multilingual':
    lang_tweets = [True] * len(lang_tweets)
tweet_train_mask = user_posted_tweet_subgraph.nodes['tweet'].data['train_mask'] & torch.tensor(lang_tweets)
tweet_val_mask = user_posted_tweet_subgraph.nodes['tweet'].data['val_mask'] & torch.tensor(lang_tweets)
tweet_test_mask = user_posted_tweet_subgraph.nodes['tweet'].data['test_mask'] & torch.tensor(lang_tweets)

plt.figure(figsize=(10, 7))
user_posted_tweet_metagraph = user_posted_tweet_subgraph.metagraph()
nx.draw_networkx(user_posted_tweet_metagraph, 
                 pos=nx.shell_layout(user_posted_tweet_metagraph), 
                 node_color='white', 
                 node_size=3000,
                 arrows=False)

# Setup dataset

In [ ]:
edges_index = torch.cat([
    user_posted_tweet_subgraph.edges(etype='posted')[0].unsqueeze(0),
    user_posted_tweet_subgraph.edges(etype='posted')[1].unsqueeze(0)
], dim=0)
data = tgData(
    x=user_posted_tweet_subgraph.nodes['tweet'].data['feat'],
    y=user_posted_tweet_subgraph.nodes['tweet'].data['label'],
    train_mask=tweet_train_mask,
    val_mask=tweet_val_mask,
    test_mask=tweet_test_mask,
    edge_index=edges_index.long())

# Train GAT

In [ ]:
MODEL_NAME = f'{LANG}_graph'
hparams = {
    'input_dim': data.num_node_features,
    'hidden_dim': 16,
    'output_dim': max(data.y).item() + 1
}

model = GAT(**hparams).double()

lr = 0.005
epochs = 400

model.train()
optimizer = optim.Adam(model.parameters(), lr=lr)

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
best_f1macro = -1

bootstrap = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    
    output = model(data.x, data.edge_index)
    loss = F.nll_loss(output[data.train_mask], data.y[data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    f1 = f1_score(output[data.train_mask], data.y[data.train_mask])
    f1macro = torch.mean(f1)

    bootstrap.update(output[data.train_mask], data.y[data.train_mask])

    if f1macro > best_f1macro:
        best_f1macro = f1macro
        torch.save(model.state_dict(), f'./data/models/{MODEL_NAME}.pth')

    if epoch % 10 == 0:
        metrics = {
            'Epoch': epoch,
            'Accuracy': accuracy(output[data.train_mask], data.y[data.train_mask]),
            'Precision': precision(output[data.train_mask], data.y[data.train_mask]),
            'Recall': recall(output[data.train_mask], data.y[data.train_mask]),
            'Stats_Score': stats_score(output[data.train_mask], data.y[data.train_mask]),
            'F1': f1,
            'F1-macro': f1macro,
            'Bootstrap': bootstrap.compute()
        }
        print(metrics)

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')

print('test')

f1 = f1_score(output[data.test_mask], data.y[data.test_mask])
f1macro = torch.mean(f1)

metrics = {
    'Accuracy': accuracy(output[data.test_mask], data.y[data.test_mask]),
    'Precision': precision(output[data.test_mask], data.y[data.test_mask]),
    'Recall': recall(output[data.test_mask], data.y[data.test_mask]),
    'Stats_Score': stats_score(output[data.test_mask], data.y[data.test_mask]),
    'F1': f1,
    'F1-macro': f1macro,
    'Bootstrap': bootstrap.compute()
}

print(metrics)

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
acc_bs = tm.BootStrapper(
    accuracy, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
f1_bs = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

acc_bs.update(output[data.test_mask], data.y[data.test_mask])
f1_bs.update(output[data.test_mask], data.y[data.test_mask])

out_acc_bs = acc_bs.compute()
out_f1_bs = f1_bs.compute()

results = {
    'Language': LANG,
    'Accuracy Bootstrap': out_acc_bs,
    'F1 Bootstrap': out_f1_bs
}

print(results)

In [ ]:
def get_all_nodes_explanation(model, explainer, data, dataset_mumin):

    feature_id_map = {
        'num_retweets': 0,
        'num_replies': 1,
        'num_quote_tweets': 2
    }

    user_df = dataset_mumin.nodes['user']
    tweet_df = dataset_mumin.nodes['tweet']
    reply_df = dataset_mumin.nodes['reply']

    reply_quoteof_tweet_df = dataset_mumin.rels[('reply', 'quote_of', 'tweet')]
    reply_quoteof_tweet_df = (reply_df.merge(reply_quoteof_tweet_df, left_index=True, right_on='src')
                            .merge(tweet_df, left_on='tgt', right_index=True)
                            .reset_index(drop=True))

    user_posted_tweet_df = dataset_mumin.rels[('user', 'posted', 'tweet')]
    user_posted_tweet_df = (user_df.merge(user_posted_tweet_df, left_index=True, right_on='src')
                            .merge(tweet_df, left_on='tgt', right_index=True)
                            .reset_index(drop=True))

    user_posted_reply_df = dataset_mumin.rels[('user', 'posted', 'reply')]
    user_posted_reply_df = (user_df.merge(user_posted_reply_df, left_index=True, right_on='src')
                            .merge(tweet_df, left_on='tgt', right_index=True)
                            .reset_index(drop=True))

    for node_idx in range(data.x.shape[0]):
        
        coefs = explainer.explain_node(node_idx, data.x, data.edge_index)
        probas = model(data.x, data.edge_index).exp()
        # fact_or_fake = 'fact' if torch.argmax(probas[node_idx]).item() == 0 else 'fake'

        if tweet_df.iloc[node_idx]['num_quote_tweets'] != 0 and \
            feature_id_map['num_quote_tweets'] not in list(np.where(coefs != 0)[0]): # get only inferences explained by quotes

            tgt_tweet_id = dataset_mumin.nodes['tweet'].iloc[node_idx]['tweet_id']

            # manual traverse
            replies_src = list(reply_quoteof_tweet_df.query(f'tweet_id_y == {tgt_tweet_id}')['src'])                      
            quoters_ids = list(user_posted_reply_df.query(f'tgt in {str(replies_src)}')['user_id'])
            quoters_posts = user_posted_tweet_df[user_posted_tweet_df['user_id'].isin(quoters_ids)]

            if not quoters_posts.empty:
                # print(fact_or_fake, quoters_ids)
                print(quoters_ids)
                # break

In [ ]:
# user_df = dataset_mumin.nodes['user']
# explainer = GraphLIME(model, hop=2, rho=0.1, cached=True)
# get_all_nodes_explanation(model, explainer, data, dataset_mumin)

# Enhance with text-based features

In [ ]:
from transformers import AutoModelForSequenceClassification

TEXT_DIM = 100

LANG_TOOL_MAP = {
    'multilingual': {
        'bertweet': AutoModel.from_pretrained('vinai/bertweet-base'),
        'tokenizer': AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=False)
    },
    'en': {
        'bertweet': AutoModel.from_pretrained('vinai/bertweet-base'),
        'tokenizer': AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=False)
    },
    'pt': {
        'bertweet': AutoModel.from_pretrained('melll-uff/bertweetbr'),
        'tokenizer': AutoTokenizer.from_pretrained('melll-uff/bertweetbr', normalization=True)
    },
    'es': {
        'bertweet': AutoModel.from_pretrained('pysentimiento/robertuito-base-cased'),
        'tokenizer': AutoTokenizer.from_pretrained('pysentimiento/robertuito-base-cased')
    }
}

bertweet = LANG_TOOL_MAP[LANG]['bertweet']
tokenizer = LANG_TOOL_MAP[LANG]['tokenizer']

# bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)
# bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)

In [ ]:
def tweetencoder(x, text_dim):
    try:
        x = bertweet(torch.tensor([tokenizer.encode(normalizeTweet(x))])).pooler_output
    except:
        # print(x)
        x = bertweet(torch.tensor([tokenizer.encode('')])).pooler_output
    return nn.Linear(768, text_dim)(x).tolist()[0]
    # return x

In [ ]:
tweet_df['text_encoding'] = str([0] * TEXT_DIM)
if LANG == 'multilingual':
    tweet_df['text_encoding'] = [tweetencoder(text, TEXT_DIM) for text in tweet_df['text']]
else:
    tweet_df['text_encoding'] = [tweetencoder(text, TEXT_DIM) if lang == LANG else str([0] * TEXT_DIM) for text,lang in zip(tweet_df['text'], tweet_df['lang'])]

In [ ]:
new_embedding_columns = [f'emb{i}' for i in range(TEXT_DIM)]

tweet_embeddings_split_df = pd.DataFrame(
    [x if not isinstance(x, str) else eval(x) for x in tweet_df['text_encoding'].tolist()],
    index=tweet_df.index,
    columns=new_embedding_columns
)
tweet_df = pd.concat([tweet_df, tweet_embeddings_split_df], axis=1)
tweet_df.dropna(inplace=True)
display(tweet_df)

In [ ]:
MODE = 'multimodal'
# MODE = 'text'

new_features_df = pd.DataFrame(index=range(user_posted_tweet_subgraph.nodes['tweet'].data['feat'].shape[0]))
new_features_df = new_features_df.join(tweet_df[new_embedding_columns])#.fillna(0)
new_features_tensor = torch.tensor(new_features_df.values).double()

mixed_features = new_features_tensor
if MODE == 'multimodal':
    mixed_features = torch.cat([user_posted_tweet_subgraph.nodes['tweet'].data['feat'], new_features_tensor], axis = 1).double()

edges_index = torch.cat([
    user_posted_tweet_subgraph.edges(etype='posted')[0].unsqueeze(0),
    user_posted_tweet_subgraph.edges(etype='posted')[1].unsqueeze(0)
], dim=0)
new_features_data = tgData(
    x=mixed_features,
    y=user_posted_tweet_subgraph.nodes['tweet'].data['label'],
    train_mask=user_posted_tweet_subgraph.nodes['tweet'].data['train_mask'] & torch.tensor(lang_tweets),
    val_mask=user_posted_tweet_subgraph.nodes['tweet'].data['val_mask'] & torch.tensor(lang_tweets),
    test_mask=user_posted_tweet_subgraph.nodes['tweet'].data['test_mask'] & torch.tensor(lang_tweets),
    edge_index=edges_index.long())

In [ ]:
hparams = {
    'input_dim': new_features_data.num_node_features,
    'hidden_dim': 16,
    'output_dim': max(new_features_data.y).item() + 1
}

MODEL_NAME = f'{LANG}_{MODE}'

model = GAT(**hparams).double()

lr = 0.005
epochs = 400

model.train()
optimizer = optim.Adam(model.parameters(), lr=lr)

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
best_f1macro = -1

bootstrap = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    
    output = model(new_features_data.x, new_features_data.edge_index)
    loss = F.nll_loss(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    f1 = f1_score(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask])
    f1macro = torch.mean(f1)

    bootstrap.update(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask])

    if f1macro > best_f1macro:
        best_f1macro = f1macro
        torch.save(model.state_dict(), f'./data/models/{MODEL_NAME}.pth')

    if epoch % 10 == 0:
        metrics = {
            'Epoch': epoch,
            'Accuracy': accuracy(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'Precision': precision(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'Recall': recall(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'Stats_Score': stats_score(output[new_features_data.train_mask], new_features_data.y[new_features_data.train_mask]),
            'F1': f1,
            'F1-macro': f1macro,
            'Bootstrap': bootstrap.compute()
        }
        print(metrics)

model.eval()

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')

print('test')

output = model(new_features_data.x, new_features_data.edge_index)

f1 = f1_score(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask])
f1macro = torch.mean(f1)

metrics = {
    'Accuracy': accuracy(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'Precision': precision(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'Recall': recall(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'Stats_Score': stats_score(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask]),
    'F1': f1,
    'F1-macro': f1macro,
    'Bootstrap': bootstrap.compute()
}

print(metrics)

In [ ]:
model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
acc_bs = tm.BootStrapper(
    accuracy, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
f1_bs = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)


output = model(new_features_data.x, new_features_data.edge_index)

acc_bs.update(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask])
f1_bs.update(output[new_features_data.test_mask], new_features_data.y[new_features_data.test_mask])

out_acc_bs = acc_bs.compute()
out_f1_bs = f1_bs.compute()

results = {
    'Language': LANG,
    'Accuracy Bootstrap': out_acc_bs,
    'F1 Bootstrap': out_f1_bs
}

print(results)

# Trustworthy?

In [ ]:
new_features_data_x_mim = torch.ones((new_features_data.x.shape[0], new_features_data.x.shape[1], 2))
new_features_data_x_mim[:,:,0] = new_features_data.x

In [ ]:
MODEL_NAME = f'{LANG}_{MODE}'

model.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model.eval()

print('test original_features')

original_features_classification_output = model(new_features_data.x, new_features_data.edge_index)
probas = original_features_classification_output.exp()
original_pred_label = torch.argmax(probas, dim=1)

## Filter features

In [ ]:
trustworthy_features_list = random.sample(range(new_features_data.x.shape[1]), k=int(new_features_data.x.shape[1] * 0.7))
untrustworthy_features_list = [i for i in range(new_features_data.x.shape[1]) if i not in trustworthy_features_list]

trustworthy_features_mask = torch.zeros(new_features_data.x.shape[1], dtype=torch.bool)
untrustworthy_features_mask = torch.zeros(new_features_data.x.shape[1], dtype=torch.bool)
# trustworthy_features_mask[trustworthy_features_list] = True
# untrustworthy_features_mask[untrustworthy_features_list] = True

trustworthy_features = new_features_data.x[:, trustworthy_features_list]
untrustworthy_features = new_features_data.x[:, untrustworthy_features_list]

trustworthy_features_data = tgData(
    x=trustworthy_features,
    y=user_posted_tweet_subgraph.nodes['tweet'].data['label'],
    train_mask=user_posted_tweet_subgraph.nodes['tweet'].data['train_mask'] & torch.tensor(lang_tweets),
    val_mask=user_posted_tweet_subgraph.nodes['tweet'].data['val_mask'] & torch.tensor(lang_tweets),
    test_mask=user_posted_tweet_subgraph.nodes['tweet'].data['test_mask'] & torch.tensor(lang_tweets),
    edge_index=edges_index.long())

In [ ]:
hparams = {
    'input_dim': trustworthy_features_data.num_node_features,
    'hidden_dim': 16,
    'output_dim': max(trustworthy_features_data.y).item() + 1
}

MODEL_NAME = f'{LANG}_{MODE}_filtered'

model_filtered = GAT(**hparams).double()

lr = 0.005
epochs = 400

model_filtered.train()
optimizer = optim.Adam(model_filtered.parameters(), lr=lr)

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
best_f1macro = -1

bootstrap = tm.BootStrapper(
    f1_score, num_bootstraps=200, sampling_strategy='multinomial', quantile=torch.tensor([0.05, 0.95])
)

for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    
    output = model_filtered(trustworthy_features_data.x, trustworthy_features_data.edge_index)
    loss = F.nll_loss(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask])
    
    loss.backward()
    optimizer.step()
    
    f1 = f1_score(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask])
    f1macro = torch.mean(f1)

    bootstrap.update(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask])

    if f1macro > best_f1macro:
        best_f1macro = f1macro
        torch.save(model_filtered.state_dict(), f'./data/models/{MODEL_NAME}.pth')

    if epoch % 10 == 0:
        metrics = {
            'Epoch': epoch,
            'Accuracy': accuracy(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask]),
            'Precision': precision(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask]),
            'Recall': recall(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask]),
            'Stats_Score': stats_score(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask]),
            'F1': f1,
            'F1-macro': f1macro,
            'Bootstrap': bootstrap.compute()
        }
        print(metrics)

model_filtered.eval()

In [ ]:
model_filtered.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model_filtered.eval()

accuracy = tm.Accuracy(task='multiclass', num_classes=2, average='none')
stats_score = tm.StatScores(task='multiclass', num_classes=2, average='none')
precision = tm.Precision(task='multiclass', num_classes=2, average='none')
recall = tm.Recall(task='multiclass', num_classes=2, average='none')
f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')

print('test')

output = model_filtered(trustworthy_features_data.x, trustworthy_features_data.edge_index)

f1 = f1_score(output[trustworthy_features_data.test_mask], trustworthy_features_data.y[trustworthy_features_data.test_mask])
f1macro = torch.mean(f1)

metrics = {
    'Accuracy': accuracy(output[trustworthy_features_data.test_mask], trustworthy_features_data.y[trustworthy_features_data.test_mask]),
    'Precision': precision(output[new_features_data.test_mask], trustworthy_features_data.y[trustworthy_features_data.test_mask]),
    'Recall': recall(output[trustworthy_features_data.test_mask], trustworthy_features_data.y[trustworthy_features_data.test_mask]),
    'Stats_Score': stats_score(output[trustworthy_features_data.test_mask], trustworthy_features_data.y[trustworthy_features_data.test_mask]),
    'F1': f1,
    'F1-macro': f1macro,
    'Bootstrap': bootstrap.compute()
}

print(metrics)

In [ ]:
MODEL_NAME = f'{LANG}_{MODE}_filtered'

model_filtered.load_state_dict(torch.load(f'./data/models/{MODEL_NAME}.pth'))
model_filtered.eval()

print('test trustworthy_features')

trustworthy_features_classification_output = model_filtered(trustworthy_features_data.x, trustworthy_features_data.edge_index)
trustworthy_pred_label = torch.argmax(trustworthy_features_classification_output.exp(), dim=1)

In [ ]:
np.where((original_pred_label == trustworthy_pred_label).numpy() == False)[0].shape, np.where((original_pred_label == trustworthy_pred_label).numpy() == True)[0].shape

In [ ]:
explainer_original = GraphLIME(model, hop=2, rho=0.1, cached=True)
explainer_filtered = GraphLIME(model_filtered, hop=2, rho=0.1, cached=True)

count_untrustworthy = 0
for i in list(np.stack(np.argwhere((original_pred_label == trustworthy_pred_label).numpy() == False),axis=1)[0]):
    
    coefs_original = explainer_original.explain_node(int(i), new_features_data.x, new_features_data.edge_index)
    coefs_filtered = explainer_filtered.explain_node(int(i), trustworthy_features_data.x, trustworthy_features_data.edge_index)
    
    count_untrustworthy += 1 if np.where(coefs_original != 0)[0].shape == np.where(coefs_filtered != 0)[0].shape else 0

count_trustworthy = int(np.stack(np.argwhere((original_pred_label == trustworthy_pred_label).numpy() == False),axis=1)[0].shape[0]) - count_untrustworthy
print(count_untrustworthy, count_trustworthy)

100 - count_untrustworthy/len(original_pred_label)*100, 100 - count_trustworthy/len(original_pred_label)*100

In [ ]:
explainer_original = GraphLIME(model, hop=2, rho=0.1, cached=True)

with open('count_untrustworthy.txt', 'w') as f:
    f.write('idx,count_untrustworthy,count_trustworthy\n')

MODEL_NAME = f'{LANG}_{MODE}_filtered'
lr = 0.005
epochs = 400
MAX_SAMPLES = 100

for sample_iter in tqdm(range(MAX_SAMPLES)):
    trustworthy_features_list = random.sample(range(new_features_data.x.shape[1]), k=int(new_features_data.x.shape[1] * 0.7))
    untrustworthy_features_list = [i for i in range(new_features_data.x.shape[1]) if i not in trustworthy_features_list]

    trustworthy_features = new_features_data.x[:, trustworthy_features_list]
    untrustworthy_features = new_features_data.x[:, untrustworthy_features_list]

    trustworthy_features_data = tgData(
        x=trustworthy_features,
        y=user_posted_tweet_subgraph.nodes['tweet'].data['label'],
        train_mask=user_posted_tweet_subgraph.nodes['tweet'].data['train_mask'] & torch.tensor(lang_tweets),
        val_mask=user_posted_tweet_subgraph.nodes['tweet'].data['val_mask'] & torch.tensor(lang_tweets),
        test_mask=user_posted_tweet_subgraph.nodes['tweet'].data['test_mask'] & torch.tensor(lang_tweets),
        edge_index=edges_index.long())

    hparams = {
        'input_dim': trustworthy_features_data.num_node_features,
        'hidden_dim': 16,
        'output_dim': max(trustworthy_features_data.y).item() + 1
    }

    model_filtered = GAT(**hparams).double()

    model_filtered.train()
    optimizer = optim.Adam(model_filtered.parameters(), lr=lr)

    f1_score = tm.classification.f_beta.F1Score(task='multiclass', num_classes=2, average='none')
    best_f1macro = -1

    for epoch in range(epochs):
        optimizer.zero_grad()
        
        output = model_filtered(trustworthy_features_data.x, trustworthy_features_data.edge_index)
        loss = F.nll_loss(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask])
        
        loss.backward()
        optimizer.step()
        
        f1 = f1_score(output[trustworthy_features_data.train_mask], trustworthy_features_data.y[trustworthy_features_data.train_mask])
        f1macro = torch.mean(f1)

        if f1macro > best_f1macro:
            best_f1macro = f1macro
            torch.save(model_filtered.state_dict(), f'./data/models/{MODEL_NAME}.pth')

    model_filtered.eval()

    trustworthy_features_classification_output = model_filtered(trustworthy_features_data.x, trustworthy_features_data.edge_index)
    trustworthy_pred_label = torch.argmax(trustworthy_features_classification_output.exp(), dim=1)

    explainer_filtered = GraphLIME(model_filtered, hop=2, rho=0.1, cached=True)

    count_untrustworthy = 0
    for i in list(np.stack(np.argwhere((original_pred_label == trustworthy_pred_label).numpy() == False),axis=1)[0]):
        
        coefs_original = explainer_original.explain_node(int(i), new_features_data.x, new_features_data.edge_index)
        coefs_filtered = explainer_filtered.explain_node(int(i), trustworthy_features_data.x, trustworthy_features_data.edge_index)
        
        count_untrustworthy += 1 if np.where(coefs_original != 0)[0].shape == np.where(coefs_filtered != 0)[0].shape else 0

    count_trustworthy = int(np.stack(np.argwhere((original_pred_label == trustworthy_pred_label).numpy() == False),axis=1)[0].shape[0]) - count_untrustworthy

    result_count_untrustworthy = 100 - count_untrustworthy/len(original_pred_label)
    result_count_trustworthy = 100 - count_trustworthy/len(original_pred_label)
    with open('count_untrustworthy.txt', 'a') as f:
        f.write(f'{sample_iter},{result_count_untrustworthy},{result_count_trustworthy}\n')

f.close()

In [ ]:
coefs_original = explainer_original.explain_node(25, new_features_data.x, new_features_data.edge_index)
coefs_filtered = explainer_filtered.explain_node(25, trustworthy_features_data.x, trustworthy_features_data.edge_index)

In [ ]:
np.where(coefs_original != 0)[0].shape == np.where(coefs_filtered != 0)[0].shape

In [ ]:
explainer = GraphLIME(model, hop=2, rho=0.1, cached=True)

for i in new_features_data.test_mask.nonzero().view(-1).tolist():
    coefs = explainer.explain_node(i, new_features_data.x, new_features_data.edge_index)
    if np.where(coefs != 0)[0].tolist() != []:
        print(i)
        print(coefs)
        break

In [ ]:
explainer = GraphLIME(model, hop=2, rho=0.1, cached=False)

probas = model(new_features_data.x, new_features_data.edge_index).exp()
coefs = explainer.explain_node(2, new_features_data.x, new_features_data.edge_index)
print(coefs, len(coefs))
print(np.where(coefs != 0))
np.where(coefs != 0)


In [ ]:
# list(tweet_df[tweet_df['lang'] == 'pt'].index)

In [ ]:
explainer = GraphLIME(model, hop=2, rho=0.1, cached=True)

# for node_idx in range(data.x.shape[0]):
#     coefs = explainer.explain_node(node_idx, data.x, data.edge_index)
#     if len(set(np.where(coefs != 0)[0]).intersection(set([3,4,5]))) != 0:
#         print(node_idx)

# try: 102 | 118 | 127
node_idx = 91

probas = model(new_features_data.x, new_features_data.edge_index).exp()
print(probas[node_idx], torch.argmax(probas[node_idx]).item())
coefs = explainer.explain_node(node_idx, new_features_data.x, new_features_data.edge_index)

print(coefs, len(coefs))
print(np.where(coefs != 0))

In [ ]:
k = 91
print(tweet_df.loc[k].text)
tweet_df.loc[k]

In [ ]:
tweet_df = dataset_mumin.nodes['tweet']
claim_df = dataset_mumin.nodes['claim']
x = dataset_mumin.rels[('tweet', 'discusses', 'claim')]
y = (tweet_df.merge(x, left_index=True, right_on='src')
                          .merge(claim_df, left_on='tgt', right_index=True)
                          .reset_index(drop=True))

y[y['tweet_id'] == 1334273990039375876]

In [ ]:
tweet_df = dataset_mumin.nodes['tweet']
reply_df = dataset_mumin.nodes['reply']
reply_quoteof_tweet_df = dataset_mumin.rels[('reply', 'reply_to', 'tweet')]
reply_quoteof_tweet_df = (reply_df.merge(quote_of_df, left_index=True, right_on='src')
                        .merge(tweet_df, left_on='tgt', right_index=True)
                        .reset_index(drop=True))

reply_quoteof_tweet_df[reply_quoteof_tweet_df['tweet_id_y'] == 1334273990039375876]

In [ ]:
from transformers_interpret import SequenceClassificationExplainer, MultiLabelClassificationExplainer
# cls_explainer = SequenceClassificationExplainer(bertweet,tokenizer)
cls_explainer = SequenceClassificationExplainer(bertweet,tokenizer)

In [ ]:
word_attributions = cls_explainer(normalizeTweet(tweet_df.loc[127].text))

In [ ]:
cls_explainer.predicted_class_name

In [ ]:
cls_explainer.visualize()

In [ ]:
word_attributions = cls_explainer(normalizeTweet('Head of Pfizer Research: Covid Vaccine is Female Sterilization – Health and Money News https://t.co/IDRLSVmkLz'))

In [ ]:
cls_explainer.visualize()

In [ ]:
bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)
F.softmax(bertweet(torch.tensor([tokenizer.encode(normalizeTweet('my favourite text'))])).logits)

In [ ]:
F.softmax(bertweet(torch.tensor([tokenizer.encode(normalizeTweet('my favourite text'))])).logits, dim=1)

In [ ]:
cls_explainer = SequenceClassificationExplainer(bertweet,tokenizer)

In [ ]:
word_attributions = cls_explainer(normalizeTweet(tweet_df.loc[102].text))
cls_explainer.visualize()

In [ ]:
dataset_mumin.nodes['claim'].loc[0]['embedding'].shape

In [ ]:
dataset_mumin.nodes['tweet']

In [ ]:
bertweet.config.id2label

In [ ]:
tweet_discusses_claim_df

In [ ]:
dataset_mumin.nodes['claim']